<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0_csv%E4%BF%9D%E5%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/株式投資')

Mounted at /content/drive


In [2]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=942ba118c1c26c3d21d64b117252d860c8e4f2a31d9ecaee346586b15b5b1f03
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
!pip install yahoo_finance_api2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3714 sha256=31e0ff3307e8d424a86ac436f7660b0aa4070118a8761136f5819bb5ff6e4059
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


# 銘柄コードから株価情報を取得する関数

In [4]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

# 東証上場の銘柄一覧を取得する関数（ダウンロードはしない）

In [5]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    # サイトからダウンロードしないようにコメントアウト
    #url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    #r = requests.get(url)
    #with open('data_j.xls', 'wb') as output:
    #    output.write(r.content)

    df = pd.read_excel("./data/data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [6]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

    close = df['Close']

    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)

    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False


    result = all(conditions)

    return result

In [18]:
def filtered_ticker(df, min_volume=None, max_price=None, min_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param min_price: 株価の最小値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    result_list = []

    # データフレームを辞書のリストに変更
    df_to_dicts = df.to_dict(orient='records')

    # 各銘柄についてループ
    for row in df_to_dicts:
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue

        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        if min_price is not None and price < min_price:
            continue

        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        result_list.append({
            '銘柄名': row['銘柄名'],
            'コード': code,
            '業種': row['業種'],
            '出来高': volume,
            '株価': price,
        })

    # 辞書のリストをデータフレームに変換
    result_df = pd.DataFrame.from_dict(result_list)

    return result_df

# CSVに保存する関数

In [8]:
import datetime as dt
import os

def save_df_as_csv(df, filename):
    """データフレームをCSVファイルとして保存する

    :param df: 対象のデータフレーム
    :param filename: 保存するcsvファイル名
    """
    # ディレクトリ名作成（YYYYMMDD_screening）
    now = dt.datetime.now().strftime("%Y%m%d")
    directory = f'{now}_screening'

    # もしディレクトリが存在しなければ、作成する
    if not os.path.exists(directory):
        os.makedirs(directory)

    # データフレームをCSVに保存する
    df.to_csv(f'{directory}/{filename}.csv', index=False)

# 東証プライムをフィルタリングしてCSVとして保存

In [12]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4262,サンドラッグ,9989,小売業
4263,サックスバー ホールディングス,9990,小売業
4264,ジェコス,9991,卸売業
4267,グローセル,9995,卸売業


In [21]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
import time
start_time = time.time()
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=500000, max_price=5000)
print(f'経過時間: {time.time() - start_time}秒')
filtered_tse_prime

Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5128]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6924]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8096]の株価情報を取得できなかったため、スキップしました。
経過時間: 229.13954663276672秒


,銘柄名,コード,業種,出来高,株価
16,ニッスイ,1332,水産・農林業,1180300,646.000000
122,三井松島ホールディングス,1518,鉱業,582500,2532.000000
159,ＩＮＰＥＸ,1605,鉱業,7249000,1597.000000
223,安藤・間,1719,建設業,673400,1089.000000
225,コムシスホールディングス,1721,建設業,500100,2813.000000
...,...,...,...,...,...
4088,エイチ・アイ・エス,9603,サービス業,5372600,2063.000000
4092,エヌ・ティ・ティ・データ,9613,情報・通信業,2693500,2108.000000
4140,ＳＣＳＫ,9719,情報・通信業,1219600,2298.000000
4191,ヤマダホールディングス,9831,小売業,3074800,431.100006


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
16,ニッスイ,1332,水産・農林業,1242400,649.299988
61,タマホーム,1419,建設業,882700,3210.000000
225,コムシスホールディングス,1721,建設業,546800,2785.000000
253,大成建設,1801,建設業,667800,4617.000000
254,大林組,1802,建設業,2224500,1176.000000
...,...,...,...,...,...
4052,東北電力,9506,電気・ガス業,3449400,895.000000
4053,四国電力,9507,電気・ガス業,1494400,992.799988
4054,九州電力,9508,電気・ガス業,2328000,924.900024
4055,北海道電力,9509,電気・ガス業,1006600,624.200012


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
16,ニッスイ,1332,水産・農林業,1242400,649.299988
225,コムシスホールディングス,1721,建設業,546800,2785.000000
254,大林組,1802,建設業,2224500,1176.000000
255,清水建設,1803,建設業,2063000,878.599976
258,長谷工コーポレーション,1808,建設業,930200,1704.000000
...,...,...,...,...,...
4052,東北電力,9506,電気・ガス業,3449400,895.000000
4053,四国電力,9507,電気・ガス業,1494400,992.799988
4054,九州電力,9508,電気・ガス業,2328000,924.900024
4055,北海道電力,9509,電気・ガス業,1006600,624.200012


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_prime, '1_東証P_UPO')

In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
266,三井住友建設,1821,建設業,960000,369.000000
392,日本Ｍ＆Ａセンターホールディングス,2127,サービス業,2422800,1054.000000
460,明治ホールディングス,2269,食料品,948000,3188.000000
480,システナ,2317,情報・通信業,1837700,318.000000
492,いちご,2337,不動産業,1064100,267.000000
527,エムスリー,2413,サービス業,3466400,3312.000000
553,エスプール,2471,サービス業,902400,548.000000
566,インフォマート,2492,サービス業,1904600,298.000000
864,ラクーンホールディングス,3031,情報・通信業,1049600,744.000000
909,東洋紡,3101,繊維製品,577300,1020.500000


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_prime = filtered_ticker(filtered_reverse_perfect_order_week_tse_prime, perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
460,明治ホールディングス,2269,食料品,948000,3188.000000
553,エスプール,2471,サービス業,902400,548.000000
909,東洋紡,3101,繊維製品,577300,1020.500000
1342,ｇｕｍｉ,3903,情報・通信業,1674100,658.000000
1420,住友化学,4005,化学,12115400,428.899994
1728,住友ファーマ,4506,医薬品,1401100,656.000000
1837,Ｚホールディングス,4689,情報・通信業,22096300,357.000000
1847,リソー教育,4714,サービス業,1086200,288.000000
1866,サイバーエージェント,4751,サービス業,3455900,1047.000000
1869,楽天グループ,4755,サービス業,22274800,547.400024


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_prime, '2_東証P_DPO')

# 東証スタンダードをフィルタリングしてCSVとして保存

In [ ]:
# 東証スタンダード市場の銘柄コードを取得
tse_standard = get_ticker_tse('standard')
tse_standard

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
34,カネコ種苗,1376,水産・農林業
37,秋川牧園,1380,水産・農林業
38,アクシーズ,1381,水産・農林業
39,ホーブ,1382,水産・農林業
40,ベルグアース,1383,水産・農林業
...,...,...,...
4257,タキヒヨー,9982,卸売業
4260,蔵王産業,9986,卸売業
4265,ヤマザワ,9993,小売業
4266,やまや,9994,小売業


In [ ]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_standard = filtered_ticker(tse_standard, min_volume=200000, max_price=5000)
filtered_tse_standard

'timestamp'
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8521]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9318]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9734]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
57,ウエストホールディングス,1407,建設業,298400,2622.0
111,中外鉱業,1491,非鉄金属,284800,30.0
120,住石ホールディングス,1514,鉱業,686200,368.0
233,中小企業ホールディングス,1757,建設業,1998700,47.0
244,アジアゲートホールディングス,1783,建設業,1231000,54.0
...,...,...,...,...,...
3934,ビジョナリーホールディングス,9263,小売業,7998700,126.0
4006,ビート・ホールディングス・リミテッド,9399,情報・通信業,8089613,5.0
4090,ウィルソン・ラーニング ワールドワイド,9610,サービス業,741900,156.0
4132,アゴーラ ホスピタリティー グループ,9704,サービス業,297400,26.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
120,住石ホールディングス,1514,鉱業,686200,368.000000
233,中小企業ホールディングス,1757,建設業,1998700,47.000000
537,シダー,2435,サービス業,281200,232.000000
563,ＪＴＰ,2488,サービス業,586200,1021.000000
670,イメージ ワン,2667,卸売業,2111100,807.000000
871,ビューティ花壇,3041,卸売業,225200,1319.000000
1000,ファンドクリエーショングループ,3266,不動産業,247300,89.000000
1190,クラウディアホールディングス,3607,繊維製品,1188000,677.000000
1227,ｅｎｉｓｈ,3667,情報・通信業,5351200,850.000000
1312,パス,3840,情報・通信業,1133000,113.000000


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_standard = filtered_ticker(filtered_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
120,住石ホールディングス,1514,鉱業,686200,368.000000
233,中小企業ホールディングス,1757,建設業,1998700,47.000000
563,ＪＴＰ,2488,サービス業,586200,1021.000000
670,イメージ ワン,2667,卸売業,2111100,807.000000
871,ビューティ花壇,3041,卸売業,225200,1319.000000
1000,ファンドクリエーショングループ,3266,不動産業,247300,89.000000
1190,クラウディアホールディングス,3607,繊維製品,1188000,677.000000
1227,ｅｎｉｓｈ,3667,情報・通信業,5351200,850.000000
1312,パス,3840,情報・通信業,1133000,113.000000
1388,イムラ,3955,パルプ・紙,247400,1245.000000


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_standard, '3_東証S_UPO')

In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
57,ウエストホールディングス,1407,建設業,298400,2622.0
244,アジアゲートホールディングス,1783,建設業,1231000,54.0
479,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,2449800,61.0
498,クシム,2345,情報・通信業,455300,391.0
705,ピクセルカンパニーズ,2743,卸売業,280100,45.0
720,新都ホールディングス,2776,卸売業,220100,87.0
724,セリア,2782,小売業,614700,2326.0
1038,レカム,3323,卸売業,23164100,95.0
1281,ブロードバンドタワー,3776,情報・通信業,235600,136.0
1299,サイバーステップ,3810,情報・通信業,342400,470.0


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_standard = filtered_ticker(filtered_reverse_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
57,ウエストホールディングス,1407,建設業,298400,2622.0
244,アジアゲートホールディングス,1783,建設業,1231000,54.0
498,クシム,2345,情報・通信業,455300,391.0
705,ピクセルカンパニーズ,2743,卸売業,280100,45.0
724,セリア,2782,小売業,614700,2326.0
1299,サイバーステップ,3810,情報・通信業,342400,470.0
1305,リミックスポイント,3825,情報・通信業,810700,237.0
2690,メディアリンクス,6659,電気機器,1120800,53.0
2722,ピクセラ,6731,電気機器,2112600,3.0
2865,大黒屋ホールディングス,6993,小売業,2175900,54.0


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_standard, '4_東証S_DPO')

# 東証グロースをフィルタリングしてCSVとして保存

In [ ]:
# 東証スタンダード市場の銘柄コードを取得
tse_grows = get_ticker_tse('grows')
tse_grows

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
55,ルーデン・ホールディングス,1400,建設業
56,エムビーエス,1401,建設業
65,Ｌｉｂ Ｗｏｒｋ,1431,建設業
70,フィット,1436,建設業
75,ニッソウ,1444,建設業
...,...,...,...
4079,プログリット,9560,サービス業
4080,グラッドキューブ,9561,サービス業
4081,ビジネスコーチ,9562,サービス業
4082,Ａｔｌａｓ Ｔｅｃｈｎｏｌｏｇｉｅｓ,9563,サービス業


In [ ]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_grows = filtered_ticker(tse_grows, min_volume=200000, max_price=5000)
filtered_tse_grows

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
400,ケアネット,2150,サービス業,1873000,989.0
406,ＦＲＯＮＴＥＯ,2158,サービス業,652500,691.0
407,ジーエヌアイグループ,2160,医薬品,314700,1118.0
481,ソフトフロントホールディングス,2321,情報・通信業,212500,109.0
490,イオレ,2334,サービス業,1447700,2035.0
...,...,...,...,...,...
3984,ｉｓｐａｃｅ,9348,サービス業,987200,1610.0
4061,リニューアブル・ジャパン,9522,電気・ガス業,1258600,788.0
4073,マイクロアド,9553,サービス業,245800,4860.0
4079,プログリット,9560,サービス業,396200,5000.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
481,ソフトフロントホールディングス,2321,情報・通信業,212500,109.0
490,イオレ,2334,サービス業,1447700,2035.0
924,海帆,3133,小売業,6489400,534.0
1171,ロコンド,3558,小売業,281400,1689.0
1233,オークファン,3674,情報・通信業,211500,736.0
1260,アプリックス,3727,情報・通信業,286800,156.0
1279,アドバンスト・メディア,3773,情報・通信業,868300,1881.0
1349,Ａｉｍｉｎｇ,3911,情報・通信業,1004000,435.0
1409,シェアリングテクノロジー,3989,情報・通信業,596900,697.0
1448,フィーチャ,4052,情報・通信業,252500,1047.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_grows = filtered_ticker(filtered_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
481,ソフトフロントホールディングス,2321,情報・通信業,212500,109.0
490,イオレ,2334,サービス業,1447700,2035.0
1171,ロコンド,3558,小売業,281400,1689.0
1233,オークファン,3674,情報・通信業,211500,736.0
1260,アプリックス,3727,情報・通信業,286800,156.0
1409,シェアリングテクノロジー,3989,情報・通信業,596900,697.0
1448,フィーチャ,4052,情報・通信業,252500,1047.0
1451,ティアンドエス,4055,情報・通信業,829100,3055.0
1495,プレイド,4165,情報・通信業,1980500,783.0
1499,ＥＮＥＣＨＡＮＧＥ,4169,情報・通信業,382300,1369.0


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_grows, '5_東証G_UPO')

In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
400,ケアネット,2150,サービス業,1873000,989.0
406,ＦＲＯＮＴＥＯ,2158,サービス業,652500,691.0
504,メディネット,2370,サービス業,1292800,69.0
540,アスカネット,2438,サービス業,202700,830.0
1285,ディー・ディー・エス,3782,情報・通信業,551200,51.0
1506,ココナラ,4176,情報・通信業,755800,386.0
1646,アクリート,4395,情報・通信業,1678800,2051.0
1680,ミンカブ・ジ・インフォノイド,4436,情報・通信業,282500,1838.0
1704,ＢＡＳＥ,4477,情報・通信業,1332200,268.0
1759,オンコセラピー・サイエンス,4564,医薬品,6653100,44.0


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_grows = filtered_ticker(filtered_reverse_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
540,アスカネット,2438,サービス業,202700,830.0
1779,リボミック,4591,医薬品,298200,164.0
1921,ステラファーマ,4888,医薬品,252400,326.0
2306,イトクロ,6049,サービス業,579600,278.0
2936,ＮｅｘＴｏｎｅ,7094,サービス業,449800,2012.0
3103,アイドマ・ホールディングス,7373,サービス業,376000,2452.0


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_grows, '6_東証G_DPO')

# 東証スタンダード・プライム選抜をCSVとして保存

In [ ]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_standard, min_volume=500000, max_price=5000)
filtered_tse_standard_2

'timestamp'
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8521]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9318]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9734]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
120,住石ホールディングス,1514,鉱業,686200,368.000000
233,中小企業ホールディングス,1757,建設業,1998700,47.000000
244,アジアゲートホールディングス,1783,建設業,1231000,54.000000
394,燦キャピタルマネージメント,2134,サービス業,15756000,31.000000
479,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,2449800,61.000000
560,出前館,2484,情報・通信業,694000,446.000000
563,ＪＴＰ,2488,サービス業,586200,1021.000000
670,イメージ ワン,2667,卸売業,2111100,807.000000
724,セリア,2782,小売業,614700,2326.000000
1038,レカム,3323,卸売業,23164100,95.000000


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2, '7_東証S_選抜')

In [ ]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_grows, min_volume=500000, max_price=5000)
filtered_tse_standard_2

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
400,ケアネット,2150,サービス業,1873000,989.0
406,ＦＲＯＮＴＥＯ,2158,サービス業,652500,691.0
490,イオレ,2334,サービス業,1447700,2035.0
504,メディネット,2370,サービス業,1292800,69.0
515,ウェッジホールディングス,2388,その他金融業,1148000,119.0
...,...,...,...,...,...
3328,イーディーピー,7794,その他製品,1172600,2506.0
3893,スカイマーク,9204,空運業,608900,1194.0
3909,マイクロ波化学,9227,サービス業,1078200,1751.0
3984,ｉｓｐａｃｅ,9348,サービス業,987200,1610.0


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2, '8_東証G_選抜')